In [1]:
import numpy as np
np.random.seed(42)
np.set_printoptions(suppress=True)
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import glob, time, os

import keras

Using TensorFlow backend.
C:\Users\Xetrov\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Xetrov\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Xetrov\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Us

In [2]:
NOTEBOOK_PATH = "C:/Users/Xetrov/Desktop/SciFair20/Code/"

# Read in data

In [10]:
f_names = glob.glob(NOTEBOOK_PATH + "IDS2017/MachineLearningCVE/*.csv")
df = pd.concat(tqdm((pd.read_csv(f) for f in f_names), total=len(f_names)))

  0%|                                                                                            | 0/8 [00:00<?, ?it/s]C:\Users\Xetrov\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\reshape\concat.py:255: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  sort=sort,
 38%|███████████████████████████████▌                                                    | 3/8 [00:02<00:04,  1.05it/s]C:\Users\Xetrov\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\reshape\concat.py:255: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  sort=sort,
100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:12<00:00,  1.59s/it]


In [11]:
df.reset_index(drop=True, inplace=True)
df.columns = list(map(lambda x: x.strip(), df.columns))

# Remove variables with no variance
remove_vars = []
for i in tqdm(range(len(df.columns))):
    col = df.columns[i]
    unique = df[col].unique()
    if len(unique) <= 1:
        remove_vars.append(col)
print(remove_vars)
df.drop(remove_vars, axis=1, inplace=True)

# Remove variables with missings and NaNs
df.drop(["Flow Bytes/s", "Flow Packets/s"], axis='columns', inplace=True)

100%|██████████████████████████████████████████████████████████████████████████████████| 79/79 [00:04<00:00, 19.21it/s]


['Bwd PSH Flags', 'Bwd URG Flags', 'Fwd Avg Bytes/Bulk', 'Fwd Avg Packets/Bulk', 'Fwd Avg Bulk Rate', 'Bwd Avg Bytes/Bulk', 'Bwd Avg Packets/Bulk', 'Bwd Avg Bulk Rate']


In [12]:
df.to_csv(NOTEBOOK_PATH + "original_df.csv", index=False)

# Normalize

In [13]:
from sklearn.preprocessing import MinMaxScaler

In [14]:
scaler = MinMaxScaler()

In [15]:
x_df = df.drop("Label", axis=1)
y_df = df.loc[:, "Label"]

In [17]:
%%time
x_norm = scaler.fit_transform(x_df, y_df)

Wall time: 14.3 s


In [19]:
pd.DataFrame(x_norm, columns=x_df.columns).describe()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,...,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06,2.830743e+06
mean,1.231629e-01,1.232140e-01,3.804712e-05,3.560461e-05,4.258158e-05,2.465873e-05,8.364220e-03,8.048884e-03,9.796892e-03,9.670788e-03,...,2.537130e-05,9.999946e-01,7.413756e-04,5.543682e-04,1.392568e-03,5.299620e-04,6.930031e-02,6.551937e-03,7.246460e-02,6.600026e-02
std,2.789903e-01,2.804478e-01,3.411356e-03,3.416626e-03,7.746968e-04,3.452708e-03,2.889544e-02,2.595241e-02,3.132396e-02,3.946155e-02,...,2.980121e-03,2.020951e-03,5.896363e-03,5.301638e-03,9.325682e-03,5.246293e-03,1.969173e-01,5.985676e-02,2.030574e-01,1.946952e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.087282e-04,1.400000e-06,4.550460e-06,3.425573e-06,9.302326e-07,0.000000e+00,2.417405e-04,0.000000e+00,1.009955e-03,0.000000e+00,...,0.000000e+00,9.999998e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.220722e-03,2.610750e-04,4.550460e-06,6.851145e-06,4.806202e-06,1.876565e-07,1.490733e-03,8.602151e-04,5.723080e-03,0.000000e+00,...,4.682591e-06,9.999998e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,6.759747e-03,2.670701e-02,1.820184e-05,1.370229e-05,1.449612e-05,7.353692e-07,3.263497e-03,1.548387e-02,8.416294e-03,3.671686e-03,...,9.365181e-06,9.999998e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [22]:
pd.DataFrame(x_norm, columns=x_df.columns).to_csv(NOTEBOOK_PATH + "IDS2017/x_normalized.csv")

# Apply ADASYN

In [ ]:
x_norm = pd.read_csv(NOTEBOOK_PATH + "IDS2017/x_normalized.csv")

In [23]:
from sklearn.model_selection import train_test_split

x_train, x_valtest, y_train, y_valtest = train_test_split(x_norm, y_df, test_size = 0.4, random_state = 42)

x_val, x_test, y_val, y_test = train_test_split(x_valtest, y_valtest, test_size = 0.5, random_state = 42)

del x_valtest 
del y_valtest

In [24]:
from imblearn.over_sampling import ADASYN

In [25]:
ada = ADASYN(random_state=42, n_jobs=-1)

In [26]:
%%time
x_train_res, y_train_res = ada.fit_resample(x_train, y_train)

RuntimeError: Not any neigbours belong to the majority class. This case will induce a NaN case with a division by zero. ADASYN is not suited for this specific dataset. Use SMOTE instead.

In [16]:
x_train_resdf = pd.DataFrame(x_train_res, columns=x_scaled.columns)

In [18]:
x_train_resdf.shape

(2727615, 68)

In [20]:
y_train_resdf = pd.DataFrame(y_train_res, columns=['IsAttack'])

In [22]:
y_train_resdf['IsAttack'].value_counts()

0    1364100
1    1363515
Name: IsAttack, dtype: int64

In [23]:
x_train_resdf.to_csv(NOTEBOOK_PATH + "IDS2017/x_adasyn_binary.csv");
y_train_resdf.to_csv(NOTEBOOK_PATH + "IDS2017/y_adasyn_binary.csv")

In [12]:
x_train_res = pd.read_csv(NOTEBOOK_PATH + "IDS2017/x_adasyn_binary.csv")

In [13]:
y_train_res = pd.read_csv(NOTEBOOK_PATH + "IDS2017/y_adasyn_binary.csv")

In [10]:
x_train_res.drop(x_train_res.columns[0], axis=1).to_csv(NOTEBOOK_PATH + "IDS2017/x_adasyn_binary.csv", index=False)

In [11]:
y_train_res.drop(y_train_res.columns[0], axis=1).to_csv(NOTEBOOK_PATH + "IDS2017/y_adasyn_binary.csv", index=False)